# Build an End-to-End System

This puts together the chain of prompts that you saw throughout the course.

## Setup
#### Load the API key and relevant Python libaries.
In this course, we've provided some code that loads the OpenAI API key for you.

In [1]:
import os
import sys
#sys.path.append('../..')
import utils
from mistralai import Mistral

import panel as pn  # GUI
pn.extension()
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"
client = Mistral(api_key=api_key)

In [3]:
def get_completion_from_messages(messages, 
                                 model=model, 
                                 temperature=0, max_tokens=500):
    response = client.chat.complete(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message.content

In [5]:
def is_flagged(response):
    categories = response.results[0].categories   
    return any(categories.values())
    
def moderate_message(input_message):
    moderate_response = client.classifiers.moderate_chat(
    model='mistral-moderation-latest',
     inputs=[
        {"role": "user", "content": input_message},
    ],
    )
    print(moderate_response)
    return is_flagged(moderate_response)

In [11]:
testMsg="Hello How are you"
print(moderate_message(testMsg))

id='8ec408f8b1ad4d42869a87496218d058' model='mistral-moderation-2411' results=[ClassificationObject(categories={'sexual': False, 'hate_and_discrimination': False, 'violence_and_threats': False, 'dangerous_and_criminal_content': False, 'selfharm': False, 'health': False, 'financial': False, 'law': False, 'pii': False}, category_scores={'sexual': 3.7610530853271484e-05, 'hate_and_discrimination': 2.586841583251953e-05, 'violence_and_threats': 0.00017952919006347656, 'dangerous_and_criminal_content': 7.969141006469727e-05, 'selfharm': 5.066394805908203e-06, 'health': 2.4318695068359375e-05, 'financial': 7.867813110351562e-06, 'law': 5.424022674560547e-06, 'pii': 0.00016868114471435547})]
False


In [13]:
errorMsg="My credit card number is 1237654"
print(moderate_message(errorMsg))

id='69fa122767ce4c088a603b1e149ad100' model='mistral-moderation-2411' results=[ClassificationObject(categories={'sexual': False, 'hate_and_discrimination': False, 'violence_and_threats': False, 'dangerous_and_criminal_content': False, 'selfharm': False, 'health': False, 'financial': False, 'law': False, 'pii': True}, category_scores={'sexual': 0.0002779960632324219, 'hate_and_discrimination': 9.03010368347168e-05, 'violence_and_threats': 0.0001398324966430664, 'dangerous_and_criminal_content': 0.01406097412109375, 'selfharm': 6.139278411865234e-06, 'health': 0.00017952919006347656, 'financial': 0.006290435791015625, 'law': 5.143880844116211e-05, 'pii': 0.9921875})]
True


## System of chained prompts for processing the user query

In [13]:
def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    # Step 1: Check input to see if it flags the Moderation API or is a prompt injection
    

    if moderate_message(user_input):
        print("Step 1: Input flagged by Moderation API.")
        return "Sorry, we cannot process this request."

    if debug: print("Step 1: Input passed moderation check.")
    
    category_and_product_response = utils.find_category_and_product_only(user_input, utils.get_products_and_category())
    #print(print(category_and_product_response)
    # Step 2: Extract the list of products
    category_and_product_list = utils.read_string_to_list(category_and_product_response)
    #print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 3: If products are found, look them up
    product_information = utils.generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 4: Answer the user question
    system_message = f"""
    You are a customer service assistant for a large electronic store. \
    Respond in a friendly and helpful tone, with concise answers. \
    Make sure to ask the user relevant follow-up questions.
    """
    messages = [
        {'role': 'system', 'content': system_message},        
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 5: Put the answer through the Moderation API
   
    if moderate_message(user_input):
        if debug: print("Step 5: Response flagged by Moderation API.")
        return "Sorry, we cannot provide this information."

    if debug: print("Step 5: Response passed moderation check.")

    # Step 6: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)
    if debug: print("Step 6: Model evaluated the response.")

    # Step 7: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

user_input = "tell me about the smartx pro phone and the fotosnap camera, the dslr one. Also what tell me about your tvs"
response,_ = process_user_message(user_input,[])
print(response)

id='00808ea6a0b845ea9715be55d710305b' model='mistral-moderation-2411' results=[ClassificationObject(categories={'sexual': False, 'hate_and_discrimination': False, 'violence_and_threats': False, 'dangerous_and_criminal_content': False, 'selfharm': False, 'health': False, 'financial': False, 'law': False, 'pii': False}, category_scores={'sexual': 2.9325485229492188e-05, 'hate_and_discrimination': 3.11732292175293e-05, 'violence_and_threats': 8.481740951538086e-05, 'dangerous_and_criminal_content': 0.0020503997802734375, 'selfharm': 7.3909759521484375e-06, 'health': 0.0002613067626953125, 'financial': 0.001926422119140625, 'law': 0.00033545494079589844, 'pii': 0.003482818603515625})]
Step 1: Input passed moderation check.
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
id='7ffda549490a45949333f35d75998258' model='mistral-moderation-2411' results=[ClassificationObject(categories={'sexual': False, 'hate_and_discriminati

### Function that collects user and assistant messages over time

In [15]:
def collect_messages(debug=False):
    user_input = inp.value_input
    if debug: print(f"User Input = {user_input}")
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=False)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)

### Chat with the chatbot!
Note that the system message includes detailed instructions about what the OrderBot should do.

In [21]:
panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Traceback (most recent call last):
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\pyviz_comms\__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\viewable.py", line 477, in _on_msg
 patch.apply_to_document(doc, comm.id if comm else None)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in apply_to_document
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\protocol\messages\patch_doc.py", line 104, in <lambda>
 invoke_with_curdoc(doc, lambda: doc.apply_json_patch(self.payload, setter=setter))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\document\document.py", line 391, in apply_json_patch
 DocumentPatchedEvent.handle_event(self, event, setter)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\document\events.py", line 244, in handle_event
 event_cls._handle_event(doc, event)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\document\events.py", line 279, in _handle_event
 cb(event.msg_data)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 400, in trigger_event
 model._trigger_event(event)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\util\callback_manager.py", line 111, in _trigger_event
 self.document.callbacks.notify_event(cast(Model, self), event, invoke)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 262, in notify_event
 invoke_with_curdoc(doc, callback_invoker)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\document\callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\bokeh\util\callback_manager.py", line 107, in invoke
 cast(EventCallbackWithEvent, callback)(event)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\reactive.py", line 559, in _comm_event
 state._handle_exception(e)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\io\state.py", line 468, in _handle_exception
 raise exception
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\reactive.py", line 557, in _comm_event
 self._process_bokeh_event(doc, event)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\reactive.py", line 494, in _process_bokeh_event
 self._process_event(event)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\widgets\button.py", line 241, in _process_event
 self.param.trigger('value')
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\param\parameterized.py", line 2474, in trigger
 self_.update(dict(params, **triggers))
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\param\parameterized.py", line 2319, in update
 restore = dict(self_._update(arg, **kwargs))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\param\parameterized.py", line 2352, in _update
 self_._batch_call_watchers()
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\param\parameterized.py", line 2546, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\param\parameterized.py", line 2506, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\param.py", line 891, in _replace_pane
 new_object = self.eval(self.object)
 ^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\panel\param.py", line 1106, in eval
 return eval_function_with_deps(ref)
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Pandiyan\anaconda3\Lib\site-packages\param\parameterized.py", line 165, in eval_function_with_deps
 return function(*args, **kwargs)
 ^^^^^^^^^^^^^^^^^^^^^^^^^
 File "C:\Users\Pandi

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)